<a href="https://colab.research.google.com/github/kumarchandan/END/blob/main/week9/2_Seq2Seq_AmbigQA_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### AmbigQA Dataset

https://nlp.cs.washington.edu/ambigqa/

a new open-domain question answering task that consists of predicting a set of question and answer pairs, where each plausible answer is associated with a disambiguated rewriting of the original question. A data set covering 14,042 open-ended QI-open questions.

#### 2 - Learning Phrase Representations using RNN Encoder-Decoder for Statistical Machine Translation¶

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import torchtext
from torchtext import data
from torchtext.data import Field, BucketIterator
from sklearn.model_selection import train_test_split

import spacy
import numpy as np

import random
import math
import time

Then set a random seed for deterministic results/reproducability.

In [2]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

Create our fields to process our data. This will append the "start of sentence" and "end of sentence" tokens as well as converting all words to lowercase.

In [3]:
TEXT = Field(tokenize='spacy', init_token='<sos>', eos_token='<eos>', lower=True)

# same number of data for question and answer so just use TEXT
fields = [('question', TEXT), ('answer', TEXT)]

Load our data.

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
! cp /content/drive/MyDrive/1-Projects/END/Week9/AmbigQA/nqopen-train.json .

In [6]:
df = pd.read_json('nqopen-train.json')
df.head()

,id,question,answer
0,-6802534628745606144,total number of death row inmates in the us,"[2,718]"
1,-8178292525996414976,big little lies season 2 how many episodes,[seven]
2,1727798345024846336,who sang waiting for a girl like you,[Foreigner]
3,-1862942322519208448,where do you cross the arctic circle in norway,[Saltfjellet]
4,-6742567993987507200,who is the main character in green eggs and ham,[Sam - I - am]


In [7]:
type(df['answer'][0])

list

In [8]:
df['answer'].str[0]

0                          2,718
1                          seven
2                      Foreigner
3                    Saltfjellet
4                   Sam - I - am
                  ...           
79163             Uhuru Kenyatta
79164              Lorenzo Music
79165    Equal Protection Clause
79166                       2001
79167              Troy Ian Hall
Name: answer, Length: 79168, dtype: object

In [9]:
df['answer'] = df['answer'].str[0]
df.head()

,id,question,answer
0,-6802534628745606144,total number of death row inmates in the us,"2,718"
1,-8178292525996414976,big little lies season 2 how many episodes,seven
2,1727798345024846336,who sang waiting for a girl like you,Foreigner
3,-1862942322519208448,where do you cross the arctic circle in norway,Saltfjellet
4,-6742567993987507200,who is the main character in green eggs and ham,Sam - I - am


In [10]:
df.shape

(79168, 3)

In [11]:
example = [data.Example.fromlist([df.question[i], df.answer[i]], fields=fields) for i in range(df.shape[0])]


In [67]:
amBigQADataset = data.Dataset(example, fields)
amBigQADataset, vars(amBigQADataset[0])

(<torchtext.data.dataset.Dataset at 0x7fc8e01584e0>,
 {'answer': ['wilhelm', 'conrad', 'röntgen'],
  'question': ['who',
   'got',
   'the',
   'first',
   'nobel',
   'prize',
   'in',
   'physics']})

In [13]:
(train, valid) = amBigQADataset.split(split_ratio=[0.85, 0.15], random_state=random.seed(SEED))
print(len(train), len(valid))

67293 11875


In [45]:
valid

We'll also print out an example just to double check they're not reversed.

In [14]:
print(vars(train.examples[0]))

{'question': ['who', 'played', 'haman', 'in', 'one', 'night', 'with', 'the', 'king'], 'answer': ['james', 'callis']}


Then create our vocabulary, converting all tokens appearing less than twice into `<unk>` tokens.

In [15]:
TEXT.build_vocab(train, min_freq = 2)
# TRG.build_vocab(train_data, min_freq = 2)

Finally, define the `device` and create our iterators.

In [16]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [17]:
BATCH_SIZE = 128

train_iterator, valid_iterator = BucketIterator.splits(
    (train, valid), 
    batch_size = BATCH_SIZE,
    sort_key = lambda x: len(x.question),
    sort_within_batch=True,
    device = device)

## Building the Seq2Seq Model

### Encoder

The encoder is similar to the previous one, with the multi-layer LSTM swapped for a single-layer GRU. We also don't pass the dropout as an argument to the GRU as that dropout is used between each layer of a multi-layered RNN. As we only have a single layer, PyTorch will display a warning if we try and use pass a dropout value to it.

Another thing to note about the GRU is that it only requires and returns a hidden state, there is no cell state like in the LSTM.

$$\begin{align*}
h_t &= \text{GRU}(e(x_t), h_{t-1})\\
(h_t, c_t) &= \text{LSTM}(e(x_t), h_{t-1}, c_{t-1})\\
h_t &= \text{RNN}(e(x_t), h_{t-1})
\end{align*}$$

From the equations above, it looks like the RNN and the GRU are identical. Inside the GRU, however, is a number of *gating mechanisms* that control the information flow in to and out of the hidden state (similar to an LSTM). Again, for more info, check out [this](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) excellent post. 

The rest of the encoder should be very familar from the last session, it takes in a sequence, $X = \{x_1, x_2, ... , x_T\}$, passes it through the embedding layer, recurrently calculates hidden states, $H = \{h_1, h_2, ..., h_T\}$, and returns a context vector (the final hidden state), $z=h_T$.

$$h_t = \text{EncoderGRU}(e(x_t), h_{t-1})$$

This is identical to the encoder of the general seq2seq model, with all the "magic" happening inside the GRU (green).

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq5.png?raw=1)

In [18]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        
        self.embedding = nn.Embedding(input_dim, emb_dim) #no dropout as only one layer!
        
        self.rnn = nn.GRU(emb_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, hidden = self.rnn(embedded) #no cell state!
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden

## Decoder

The decoder is where the implementation differs significantly from the previous model and we alleviate some of the information compression.

Instead of the GRU in the decoder taking just the embedded target token, $d(y_t)$ and the previous hidden state $s_{t-1}$ as inputs, it also takes the context vector $z$. 

$$s_t = \text{DecoderGRU}(d(y_t), s_{t-1}, z)$$

Note how this context vector, $z$, does not have a $t$ subscript, meaning we re-use the same context vector returned by the encoder for every time-step in the decoder. 

Before, we predicted the next token, $\hat{y}_{t+1}$, with the linear layer, $f$, only using the top-layer decoder hidden state at that time-step, $s_t$, as $\hat{y}_{t+1}=f(s_t^L)$. Now, we also pass the embedding of current token, $d(y_t)$ and the context vector, $z$ to the linear layer.

$$\hat{y}_{t+1} = f(d(y_t), s_t, z)$$

Thus, our decoder now looks something like this:

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq6.png?raw=1)

Note, the initial hidden state, $s_0$, is still the context vector, $z$, so when generating the first token we are actually inputting two identical context vectors into the GRU.

How do these two changes reduce the information compression? Well, hypothetically the decoder hidden states, $s_t$, no longer need to contain information about the source sequence as it is always available as an input. Thus, it only needs to contain information about what tokens it has generated so far. The addition of $y_t$ to the linear layer also means this layer can directly see what the token is, without having to get this information from the hidden state. 

However, this hypothesis is just a hypothesis, it is impossible to determine how the model actually uses the information provided to it (don't listen to anyone that says differently). Nevertheless, it is a solid intuition and the results seem to indicate that this modifications are a good idea!

Within the implementation, we will pass $d(y_t)$ and $z$ to the GRU by concatenating them together, so the input dimensions to the GRU are now `emb_dim + hid_dim` (as context vector will be of size `hid_dim`). The linear layer will take $d(y_t), s_t$ and $z$ also by concatenating them together, hence the input dimensions are now `emb_dim + hid_dim*2`. We also don't pass a value of dropout to the GRU as it only uses a single layer.

`forward` now takes a `context` argument. Inside of `forward`, we concatenate $y_t$ and $z$ as `emb_con` before feeding to the GRU, and we concatenate $d(y_t)$, $s_t$ and $z$ together as `output` before feeding it through the linear layer to receive our predictions, $\hat{y}_{t+1}$.

In [19]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        self.output_dim = output_dim
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU(emb_dim + hid_dim, hid_dim)
        
        self.fc_out = nn.Linear(emb_dim + hid_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, context):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #context = [n layers * n directions, batch size, hid dim]
        
        #n layers and n directions in the decoder will both always be 1, therefore:
        #hidden = [1, batch size, hid dim]
        #context = [1, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
                
        emb_con = torch.cat((embedded, context), dim = 2)
            
        #emb_con = [1, batch size, emb dim + hid dim]
            
        output, hidden = self.rnn(emb_con, hidden)
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #seq len, n layers and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        output = torch.cat((embedded.squeeze(0), hidden.squeeze(0), context.squeeze(0)), 
                           dim = 1)
        
        #output = [batch size, emb dim + hid dim * 2]
        
        prediction = self.fc_out(output)
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden

## Seq2Seq Model

Putting the encoder and decoder together, we get:

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq7.png?raw=1)

Again, in this implementation we need to ensure the hidden dimensions in both the encoder and the decoder are the same.

Briefly going over all of the steps:
- the `outputs` tensor is created to hold all predictions, $\hat{Y}$
- the source sequence, $X$, is fed into the encoder to receive a `context` vector
- the initial decoder hidden state is set to be the `context` vector, $s_0 = z = h_T$
- we use a batch of `<sos>` tokens as the first `input`, $y_1$
- we then decode within a loop:
  - inserting the input token $y_t$, previous hidden state, $s_{t-1}$, and the context vector, $z$, into the decoder
  - receiving a prediction, $\hat{y}_{t+1}$, and a new hidden state, $s_t$
  - we then decide if we are going to teacher force or not, setting the next input as appropriate (either the ground truth next token in the target sequence or the highest predicted next token)

In [20]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is the context
        context = self.encoder(src)
        
        #context also used as the initial hidden state of the decoder
        hidden = context
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden state and the context state
            #receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(input, hidden, context)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1

        return outputs

# Training the Seq2Seq Model

The rest of this session is very similar to the previous one. 

We initialise our encoder, decoder and seq2seq model (placing it on the GPU if we have one). As before, the embedding dimensions and the amount of dropout used can be different between the encoder and the decoder, but the hidden dimensions must remain the same.

In [21]:
INPUT_DIM = len(TEXT.vocab)
OUTPUT_DIM = len(TEXT.vocab)
ENC_EMB_DIM = 128
DEC_EMB_DIM = 128
HID_DIM = 256
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, DEC_DROPOUT)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Seq2Seq(enc, dec, device).to(device)

Next, we initialize our parameters. The paper states the parameters are initialized from a normal distribution with a mean of 0 and a standard deviation of 0.01, i.e. $\mathcal{N}(0, 0.01)$. 

It also states we should initialize the recurrent parameters to a special initialization, however to keep things simple we'll also initialize them to $\mathcal{N}(0, 0.01)$.

In [22]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean=0, std=0.01)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(21693, 128)
    (rnn): GRU(128, 256)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(21693, 128)
    (rnn): GRU(384, 256)
    (fc_out): Linear(in_features=640, out_features=21693, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

We print out the number of parameters.

Even though we only have a single layer RNN for our encoder and decoder we actually have **more** parameters  than the last model. This is due to the increased size of the inputs to the GRU and the linear layer. However, it is not a significant amount of parameters and causes a minimal amount of increase in training time (~3 seconds per epoch extra).

In [23]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 20,248,125 trainable parameters


We initiaize our optimizer.

In [24]:
optimizer = optim.Adam(model.parameters())

We also initialize the loss function, making sure to ignore the loss on `<pad>` tokens.

In [25]:
TEXT_PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TEXT_PAD_IDX)

We then create the training loop...

In [26]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.question
        trg = batch.answer
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

...and the evaluation loop, remembering to set the model to `eval` mode and turn off teaching forcing.

In [27]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.question
            trg = batch.answer

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

We'll also define the function that calculates how long an epoch takes.

In [28]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

Then, we train our model, saving the parameters that give us the best validation loss.

In [29]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 44s
	Train Loss: 5.638 | Train PPL: 280.829
	 Val. Loss: 4.939 |  Val. PPL: 139.597
Epoch: 02 | Time: 0m 44s
	Train Loss: 4.881 | Train PPL: 131.787
	 Val. Loss: 4.758 |  Val. PPL: 116.571
Epoch: 03 | Time: 0m 44s
	Train Loss: 4.577 | Train PPL:  97.182
	 Val. Loss: 4.635 |  Val. PPL: 103.051
Epoch: 04 | Time: 0m 44s
	Train Loss: 4.306 | Train PPL:  74.167
	 Val. Loss: 4.544 |  Val. PPL:  94.090
Epoch: 05 | Time: 0m 44s
	Train Loss: 4.048 | Train PPL:  57.258
	 Val. Loss: 4.501 |  Val. PPL:  90.063
Epoch: 06 | Time: 0m 44s
	Train Loss: 3.752 | Train PPL:  42.617
	 Val. Loss: 4.465 |  Val. PPL:  86.918
Epoch: 07 | Time: 0m 44s
	Train Loss: 3.451 | Train PPL:  31.539
	 Val. Loss: 4.445 |  Val. PPL:  85.218
Epoch: 08 | Time: 0m 44s
	Train Loss: 3.141 | Train PPL:  23.136
	 Val. Loss: 4.496 |  Val. PPL:  89.651
Epoch: 09 | Time: 0m 44s
	Train Loss: 2.864 | Train PPL:  17.532
	 Val. Loss: 4.542 |  Val. PPL:  93.886
Epoch: 10 | Time: 0m 44s
	Train Loss: 2.585 | Train PPL

In [77]:
test_df = pd.read_json('nqopen-test.json')
test_df.head()

,id,question,answer
0,-3290814144789249536,who got the first nobel prize in physics,[Wilhelm Conrad Röntgen]
1,8851020722386421760,when is the next deadpool movie being released,"[May 18 , 2018]"
2,-8817510910624891904,which mode is used for short wave broadcast se...,"[Olivia, MFSK]"
3,955374967862684288,the south west wind blows across nigeria between,[till September]
4,-4340755100872459776,what does hp mean in war and order,[hit points or health points]


In [78]:
test_df['answer'] = test_df['answer'].str[0]
test_df.head()

,id,question,answer
0,-3290814144789249536,who got the first nobel prize in physics,Wilhelm Conrad Röntgen
1,8851020722386421760,when is the next deadpool movie being released,"May 18 , 2018"
2,-8817510910624891904,which mode is used for short wave broadcast se...,Olivia
3,955374967862684288,the south west wind blows across nigeria between,till September
4,-4340755100872459776,what does hp mean in war and order,hit points or health points


In [79]:
example_test = [data.Example.fromlist([test_df.question[i], test_df.answer[i]], fields=fields) for i in range(test_df.shape[0])]

In [80]:
amBigQADatasetTest = data.Dataset(example_test, fields)
amBigQADatasetTest

In [81]:
vars(amBigQADatasetTest[0])

{'answer': ['wilhelm', 'conrad', 'röntgen'],
 'question': ['who', 'got', 'the', 'first', 'nobel', 'prize', 'in', 'physics']}

In [82]:
test_iterator = torchtext.data.BucketIterator( # since only test data, no need of split
    amBigQADatasetTest, 
    batch_size = BATCH_SIZE,
    sort_key = lambda x: len(x.question),
    sort_within_batch=True,
    device = device, )

In [83]:
for batch in test_iterator:
  print(batch)
  break


[torchtext.data.batch.Batch of size 128]
	[.question]:[torch.cuda.LongTensor of size 12x128 (GPU 0)]
	[.answer]:[torch.cuda.LongTensor of size 7x128 (GPU 0)]


Finally, we test the model on the test set using these "best" parameters.

In [84]:
model.load_state_dict(torch.load('tut2-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 6.343 | Test PPL: 568.274 |
